# 氏名漢字カナ突合モデル

漢字カナ突合モデルはJava17で記述されている。Mavenを用いて漢字カナ突合モデルをビルドする。
仕様等については[README](./README.md)を参照のこと。


## 必要なモジュール

- JDK 17
- Maven 3




## 事前準備
- バージョン
pom.xmlのバージョン番号を書き換える

```xml
    <groupId>jp.go.digital.kanjikana</groupId>
    <artifactId>kanjikana_core_oss</artifactId>
    <version>1.6o</version>   // ここを書き換えるとコンパイル後のJarファイル名にもこのバージョン番号が入る

```

- ファイル
[dict](../dict)で作成した，辞書ファイル及び，[train](../train)で作成したAIモデルのファイルを，Javaのリソースとして使用するために，src/main/resources以下にファイルをコピーする。

In [3]:
!mvn clean antrun:run


[INFO] Scanning for projects...
[INFO] 
[INFO] -------------< jp.go.digital.kanjikana:kanjikana_core_oss >-------------
[INFO] Building kanjikana_core_oss 1.6o
[INFO]   from pom.xml
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- clean:3.2.0:clean (default-clean) @ kanjikana_core_oss ---
[INFO] Deleting /Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/core/target
[INFO] 
[INFO] --- antrun:3.1.0:run (default-cli) @ kanjikana_core_oss ---
[INFO] Executing tasks
[INFO]      [copy] Copying 1 file to /Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/core/src/main/resources/dict
[INFO] Executed tasks
[INFO] ------------------------------------------------------------------------
[INFO] BUILD SUCCESS
[INFO] ------------------------------------------------------------------------
[INFO] Total time:  4.140 s
[INFO] Finished at: 2024-12-24T15:08:03+09:00
[INFO] ----------------------------------------------------------------------

## コンパイルする


In [4]:
!mvn compile assembly:single


[INFO] Scanning for projects...
[INFO] 
[INFO] -------------< jp.go.digital.kanjikana:kanjikana_core_oss >-------------
[INFO] Building kanjikana_core_oss 1.6o
[INFO]   from pom.xml
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- resources:3.2.0:resources (default-resources) @ kanjikana_core_oss ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] Using 'UTF-8' encoding to copy filtered properties files.
[INFO] Copying 28 resources
[INFO] The encoding used to copy filtered properties files have not been set. This means that the same encoding will be used to copy filtered properties files as when copying other filtered resources. This might not be what you want! Run your build with --debug to see which files might be affected. Read more at https://maven.apache.org/plugins/maven-resources-plugin/examples/filtering-properties-files.html
[INFO] 
[INFO] --- compiler:3.13.0:compile (default-compile) @ kanjikana_core_oss ---
[

## 実行する

漢字姓名とカナ姓名が入力されたファイルをCSVもしくはTSV形式で用意する。漢字姓名とカナ姓名以外のフィールドが含まれていても良い。
なお，VMパラメタとして`-Xmx4096M`を指定し，４Gのメモリを確保すること。
### 実行クラス
jp.go.digital.kanjikana.core.executor.match.KanjiKanaMatchMain
### パラメタ
- infile
入力ファイル。行に漢字姓名とカナ姓名を含むファイルで，TSVもしくはCSV形式で作成しること。また，ヘッダがあっても良くヘッダがある場合には`has_header`パラメタを指定する。また，漢字姓名とカナ姓名の列を`kanji_idx`と`kana_idx`パラメタで指定する
- okfile
漢字姓名とカナ姓名が一致と判断されたレコードが保存されるファイル
- ngfile
漢字姓名とカナ姓名が不一致と判断されたレコードが保存されるファイル
- logfile
入力ファイルの各行を実行する際のログファイル
- kanji_idx
infileのファイルの漢字姓名の列番号，0から始まる
- kana_idx
infileのファイルのカナ姓名の列番号，0から始まる
- sep
infileの各列の区切り文字を指定する。csvもしくはtsvを指定する。
- thread_num
infileに大量の行が含まれる場合には，本プログラムをスレッドで多重化して計算することができる。`thread_num`に1を指定すると（デフォルト），シングルスレッドで実行され，それ以上を値を指定するとその数だけスレッドで計算される。スレッドでケインさんした場合には，`okfile`と`ngfile`，`logfile`の末尾にスレッド番号が付与されたファイルが作成される
- has_header
infileにヘッダ業がある場合には`true`を指定する。デフォルトは`false`であり，infileにヘッダ行は含まれない
- strategy
漢字姓名とカナ姓名の突合モデルをどれを使うかを指定する。BASIC, ONLY_AI, ONLY_DICT, AI, ENSEMBLEから選択し，セットする。    

  |モデル|内容|
  |-----|---|
  |ENSEMBLE||
  |BASIC|姓名辞書，異体字辞書との突合のみ|
  |ONLY_AI|AIモデルとの突合のみ|
  |ONLY_DICT|姓名辞書及び単漢字辞書，異体字辞書との突合のみ|
  |AI|BASIC + ONLY_AI|
  

In [5]:
# 出力結果がng.txtとok.txtに出力される
!java -Xmx4096M -Dlog4j.configurationFile=log4j2.xml -classpath target/kanjikana_core_oss-1.6o-jar-with-dependencies.jar jp.go.digital.kanjikana.core.executor.match.KanjiKanaMatchMain --infile input.txt --okfile ok.txt --ngfile ng.txt --logfile log.txt --kanji_idx 1 --kana_idx 2 --strategy ENSEMBLE


16:32:43.686 [main] ERROR jp.go.digital.kanjikana.core.executor.match.KanjiKanaMatchRunner - [Ljava.lang.StackTraceElement;@7f284218
16:32:43.688 [main] ERROR jp.go.digital.kanjikana.core.executor.match.KanjiKanaMatchRunner - 
16:32:44.447 [main] DEBUG ai.djl.repository.zoo.DefaultModelZoo -- Scanning models in repo: class ai.djl.repository.SimpleRepository, /var/folders/nd/8rmfsv815m76kpkw1wjlgddc0000gn/T/model_5655324545920443220.tmp
16:32:44.452 [main] DEBUG ai.djl.engine.Engine -- Registering EngineProvider: PyTorch
16:32:44.452 [main] DEBUG ai.djl.engine.Engine -- Found default engine: PyTorch
16:32:44.535 [main] DEBUG ai.djl.repository.zoo.ModelZoo -- Loading model with Criteria:
	Application: UNDEFINED
	Input: class ai.djl.ndarray.NDList
	Output: class ai.djl.ndarray.NDList
	Engine: PyTorch
	ModelZoo: ai.djl.localmodelzoo
	No translator supplied

16:32:44.535 [main] DEBUG ai.djl.repository.zoo.ModelZoo -- Searching model in specified model zoo: ai.djl.localmodelzoo
16:32:44.535 